# An intro to PhiData

In [2]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.tools.yfinance import YFinanceTools

In [3]:
agent = Agent(
  model=OpenAIChat(id="gpt-4o"),
  tools=[
    YFinanceTools(
      stock_price=True, 
      analyst_recommendations=True, 
      stock_fundamentals=True
    )
  ],
  instructions="Use tables to display data.",
  show_tool_calls=True,
  markdown=True,
)

In [4]:
agent.print_response(
  message="""
  Tell me which of MDB and ESTC is a better pick.
  Summarize and show me in tables.
  """,
  stream=True
)

Output()

## Using Ollama

In [5]:
from phi.model.ollama import Ollama

In [6]:
agent = Agent(
  model=Ollama(id="llama3.2"),
  tools=[
    YFinanceTools(
      stock_price=True, 
      analyst_recommendations=True, 
      stock_fundamentals=True
    )
  ],
  instructions="Use tables to display data.",
  show_tool_calls=True,
  markdown=True,
)

In [7]:
agent.print_response(
  message="""
  Tell me which of MDB and ESTC is a better pick.
  Summarize and show me in tables.
  """,
  stream=True
)

Output()

## Vector database

In [8]:
from phi.vectordb.lancedb import LanceDb, SearchType
from phi.knowledge.text import TextKnowledgeBase
from phi.embedder.ollama import OllamaEmbedder
from pathlib import Path

In [9]:
vector_db=LanceDb(
  table_name="articles",
  uri="tmp/lancedb",
  search_type=SearchType.vector,
  embedder=OllamaEmbedder(model="mxbai-embed-large"),
)

In [10]:
knowledge_base = TextKnowledgeBase(
  path=Path("docs"),
  vector_db=vector_db,
  num_documents=5,
)

In [11]:
knowledge_base.load(recreate=False)

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: docs/mongo.txt

INFO     Added 1 documents to knowledge base

## Knowledge agent

In [12]:
knowledge_agent = Agent(
  knowledge=knowledge_base,
  search_knowledge=True,
)

In [13]:
knowledge_agent.print_response(
  message="How is MongoDB doing?", 
  stream=True
)

Output()

## A team of agents

In [14]:
agent_team = Agent(
  team=[agent, knowledge_agent],
  instructions=["Always include sources", "Use tables to display data"],
  show_tool_calls=True,
  markdown=True,
)

In [15]:
agent_team.print_response(
  message="How is MongoDB doing? In terms of analysts and articles", 
  stream=True
)

Output()